# CartPole Game 🕹️

## Packages

In [1]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


## Class Agent

In [3]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


## Main

In [3]:
import sys, os
os.chdir(r'C:\Users\Quentin\Documents\5 GMM Projet\save')  # to change the current directory

In [4]:
EPISODES = 100

env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
#agent.load("cartpole-dqn.h5")
done = False
batch_size = 32

for e in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(500):
        env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}"
                  .format(e, EPISODES-1, time, agent.epsilon))
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
    if e % 10 == 0:
        agent.save("cartpole-dqn.h5")
env.close()

episode: 0/99, score: 62, e: 0.99
episode: 1/99, score: 85, e: 0.94
episode: 2/99, score: 22, e: 0.93
episode: 3/99, score: 12, e: 0.93
episode: 4/99, score: 41, e: 0.91
episode: 5/99, score: 16, e: 0.9
episode: 6/99, score: 30, e: 0.89
episode: 7/99, score: 22, e: 0.88
episode: 8/99, score: 17, e: 0.87
episode: 9/99, score: 19, e: 0.86
episode: 10/99, score: 9, e: 0.86
episode: 11/99, score: 19, e: 0.85
episode: 12/99, score: 19, e: 0.84
episode: 13/99, score: 27, e: 0.83
episode: 14/99, score: 14, e: 0.83
episode: 15/99, score: 28, e: 0.81
episode: 16/99, score: 12, e: 0.81
episode: 17/99, score: 54, e: 0.79
episode: 18/99, score: 15, e: 0.78
episode: 19/99, score: 11, e: 0.78
episode: 20/99, score: 15, e: 0.77
episode: 21/99, score: 10, e: 0.77
episode: 22/99, score: 31, e: 0.76
episode: 23/99, score: 17, e: 0.75
episode: 24/99, score: 12, e: 0.75
episode: 25/99, score: 7, e: 0.74
episode: 26/99, score: 8, e: 0.74
episode: 27/99, score: 17, e: 0.73
episode: 28/99, score: 14, e: 0.73